In [66]:
%reload_ext autoreload
%autoreload 2

In [67]:
import sys
sys.path.append('../DataRequest/')

# Amazon ML + Sinergise Blog


## Data Acquisition

This notebook shows how to obtain raw data (Sentinel 2 satellite images) using Sentinel Hub and ground truth (polygons of tulip fields) from Geopedia. At the moment Geopedia contains only data from 2016. 2017 data and other crop types (i.e. other flower types) can be easily added.

###### Required WMS instance

The instance below was created by Sinergise for the purpuse of this blog. It was created on Jed's name using his email address (jed@amazon.de). Use this instance for accessing Sentinel 2 data via Sentinel Hub.

In [68]:
WMS_INSTANCE = '71513b0b-264d-494a-b8c4-c3c36433db28'

In [69]:
sentinel_hub_wms='https://services.sentinel-hub.com/ogc/wms/'+WMS_INSTANCE

In [70]:
geopedia_layers = {'tulip_field_2016':'ttl1904', 'tulip_field_2017':'ttl1905', 'arable_land_2017':'ttl1917'}
s2_wms_layers = ['TRUE_COLOR', 'ALL_BANDS']

###### Required packages

For interactive display of Sentinel 2 images with tulip fields overlay a dev version of ipyleaflet is required, which can be found here: https://github.com/ellisonbg/ipyleaflet

To install it run:

    $ pip install ipyleaflet
$ jupyter nbextension enable --py --sys-prefix ipyleaflet

also install pyproj, opencv, tifffile

## Interactive display

Create map and center it around Den Helder, Netherlands. You can zoom in/out and change the view using your mouse. 

In [71]:
from ipyleaflet import Map, WMSLayer, DrawControl, Rectangle

In [72]:
zoom_level = 14

In [73]:
import math
earth_radius = 6372.7982e3
pixel_size = 2* math.pi * earth_radius * math.cos(math.radians(0))/2**(zoom_level+8)
print('Pixel dimension at zoom level %d equals %1.2f m.'%(zoom_level,pixel_size))

Pixel dimension at zoom level 14 equals 9.55 m.


In [74]:
m = Map(center=[52.9255665659715, 4.754333496093751], zoom=zoom_level, layout=dict(width='512px', height='512px'))

### Create DrawController with callback to cover a polygon with sliding patches

We will be using 256x256px images. With Sentinel-2's resolution of 10m/px, it makes sense to have our images cover 2560x2560m. This way we make full use of the satellite's capabilities. In addition, we'll have each two images overlap by 10% of their side, so stride = 2560 - 256 = 2304. 

In [75]:
# PolygonSlidingWindow is a callable class following the DrawController on_draw() API
from gather_data import PolygonSlidingWindow, GeoJsonSaver
poly_sw_cb = PolygonSlidingWindow(box_width=2560, box_height=2560, stride_x=2304, stride_y=2304)

In [76]:
# A callback of PolygonSlidingWindow to display the patches on the map as they are created
# -> Patches with green border are for training; Patches with red border are for testing
tiles = [] # save layer handles for later clean-up
def draw_patches_on_map(ipy_map):
    def aux(self, patch):
        colors = { None: '#00F', 'train': '#0F0', 'test': '#F00' }
        rect_kwargs = {'bounds': patch.box, 'weight': 1, 'color': '#00F', 'opacity': 0.2, 'fill_opacity': 0.2}
        if patch.contained:
            pg = Rectangle(fill_color=colors[patch.set], **rect_kwargs)
            ipy_map.add_layer(pg)
            tiles.append(pg)
    return aux
poly_sw_cb.register_callback(draw_patches_on_map(m))

In [77]:
# Create DrawControl with our callback on polygon creation
dc = DrawControl()
dc.on_draw(poly_sw_cb)

Add another callback to save the created polygons so they can be reused 

In [78]:
saver = GeoJsonSaver('downloads/sample_region_2/')
dc.on_draw(saver)    

In [61]:
poly_sw_cb.set_mode('train')
poly_sw_cb.load_polygons_from_folder('downloads/sample_region_2/')

DEBUG:root:Found 0 json files


In [35]:
# Check how many patches are contained in the defined polygons
patches = list(poly_sw_cb.patches.values())[0]
patches = [item for sublist in patches for item in sublist]
len(patches)

4665

In [84]:
m.add_control(dc)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[52.9255665659715, 4.754333496093751], controls=(DrawControl(layer=FeatureGroup(), polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}),), layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']),), layout=Layout(height='512px', width='512px'), options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=14)

In [79]:
poly_sw_cb.set_mode('train')

In [ ]:
poly_sw_cb.set_mode('test')

In [17]:
{ k:len([ p for v in poly for p in v if p.contained ]) for k,poly in poly_sw_cb.patches.items() }

{'train': 137}

#### Add Sentinel 2 layer to the map

In [89]:
s2_layer = WMSLayer(url='https://services.sentinel-hub.com/v1/wms/'+WMS_INSTANCE, layers='TRUE_COLOR', tile_size=512)

In [90]:
m.add_layer(s2_layer)

#### Add ground truth layer (all tulip fields from 2016)

In [91]:
tulips = WMSLayer(url='http://service.geopedia.world/wms/ml_aws', layers=geopedia_layers['tulip_field_2016'], tile_size=512, format='image/png', version='1.3.0', TRANSPARENT=True, opacity=0.4)

In [92]:
m.add_layer(tulips)

#### openstreetmap

In [87]:
wms = WMSLayer(url='https://ows.terrestris.de/osm/service', layers='OSM-Overlay-WMS', tile_size=512)

In [88]:
m.add_layer(wms)

#### Clean-up layers

In [43]:
list(m.remove_layer(pg) for pg in tiles);

## Run downloader

In [93]:
import uuid
#root_dir = 'images/{}/'.format(uuid.uuid4())
root_dir = 'downloads/'

In [89]:
# A callback to display download progress on map
def show_progress_on_map(ipy_map, color):
    def aux(patch):
        if patch.contained:
            colors = { None: '#00F', 'train': '#0F0', 'test': '#F00' }
            rect_kwargs = {'bounds': patch.bounds, 'weight': 1, 'color': colors[patch.set], 'opacity': 0.2, 'fill_opacity': 0.2}
            pg = Rectangle(fill_color=color, **rect_kwargs)
            ipy_map.add_layer(pg)
            tiles.append(pg)
    return aux

In [94]:
from DataRequest import TulipFieldRequest, S2Request
# A function to download data for all the patches in the sliding window
from gather_data import BatchDownloader

In [95]:
poly_sw_cb.set_mode('train')
downloader_train_labels = BatchDownloader(root_dir+'sample_region_2/', poly_sw_cb, TulipFieldRequest, (), {'width':256, 'height':256, 'crs':3857, 'layer':geopedia_layers['tulip_field_2016']})

In [96]:
downloader_train_labels.download_data()

DEBUG:root:Adding patch 52.678145335053124,5.607234184146118 --> 52.69208592394724,5.630231055419578 to queue ([(624194.4541377827, 6923684.013462493), (626754.4541377827, 6926244.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.690692065307466,5.607234184146118 --> 52.70462864937853,5.630231055419578 to queue ([(624194.4541377827, 6925988.013462493), (626754.4541377827, 6928548.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.70323519119693,5.607234184146118 --> 52.717167770927745,5.630231055419578 to queue ([(624194.4541377827, 6928292.013462493), (626754.4541377827, 6930852.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.71577471315601,5.607234184146118 --> 52.72970328902998,5.630231055419578 to queue ([(624194.4541377827, 6930596.013462493), (626754.4541377827, 6933156.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.7283106316

DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.765896770397845,5.66932573658446 --> 52.77980933569124,5.692322607857919 to queue ([(631106.4541377827, 6939812.013462493), (633666.4541377827, 6942372.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.77841827924198,5.66932573658446 --> 52.79232684310505,5.692322607857919 to queue ([(631106.4541377827, 6942116.013462493), (633666.4541377827, 6944676.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.79093618677448,5.66932573658446 --> 52.804840749694414,5.692322607857919 to queue ([(631106.4541377827, 6944420.013462493), (633666.4541377827, 6946980.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.80345049343377,5.66932573658446 --> 52.81735105589834,5.692322607857919 to queue ([(631106.4541377827, 6946724.013462493), (633666.4541377827, 6949284.013462493)])
DEBUG:root:There are 1 urls queued for dow

DEBUG:root:Adding patch 52.77841827924198,5.731417289022801 --> 52.79232684310505,5.75441416029626 to queue ([(638018.4541377827, 6942116.013462493), (640578.4541377827, 6944676.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.79093618677448,5.731417289022801 --> 52.804840749694414,5.75441416029626 to queue ([(638018.4541377827, 6944420.013462493), (640578.4541377827, 6946980.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.80345049343377,5.731417289022801 --> 52.81735105589834,5.75441416029626 to queue ([(638018.4541377827, 6946724.013462493), (640578.4541377827, 6949284.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.678145335053124,5.752114473168914 --> 52.69208592394724,5.775111344442374 to queue ([(640322.4541377827, 6923684.013462493), (642882.4541377827, 6926244.013462493)])
DEBUG:root:There are 1 urls queued for download
DEBUG:root:Adding patch 52.69069206530746

send: b'GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=624194.4541377827,6923684.013462493,626754.4541377827,6926244.013462493 HTTP/1.1\r\nHost: service.geopedia.world\r\nUser-Agent: python-requests/2.18.4\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Expires header: X-GPD-Session header: Cache-Control header: Date header: Vary header: Content-Length header: Content-Type header: Last-Modified header: Age header: Via header: Connection Downloaded from http://service.geopedia.world/wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=624194.4541377827,6923684.013462493,626754.4541377827,6926244.013462493
send: b'GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.

DEBUG:urllib3.connectionpool:http://service.geopedia.world:80 "GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=624194.4541377827,6925988.013462493,626754.4541377827,6928548.013462493 HTTP/1.1" 200 1331
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): service.geopedia.world
DEBUG:urllib3.connectionpool:http://service.geopedia.world:80 "GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=624194.4541377827,6928292.013462493,626754.4541377827,6930852.013462493 HTTP/1.1" 200 1100
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): service.geopedia.world


...

DEBUG:urllib3.connectionpool:http://service.geopedia.world:80 "GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=642626.4541377827,6937508.013462493,645186.4541377827,6940068.013462493 HTTP/1.1" 200 1285
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): service.geopedia.world


reply: 'HTTP/1.1 200 OK\r\n'
header: Expires header: X-GPD-Session header: Cache-Control header: Date header: Vary header: Content-Length header: Content-Type header: Last-Modified header: Age header: Via header: Connection Downloaded from http://service.geopedia.world/wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=642626.4541377827,6937508.013462493,645186.4541377827,6940068.013462493
send: b'GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493 HTTP/1.1\r\nHost: service.geopedia.world\r\nUser-Agent: python-requests/2.18.4\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'


DEBUG:urllib3.connectionpool:http://service.geopedia.world:80 "GET /wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493 HTTP/1.1" 200 4094


reply: 'HTTP/1.1 200 OK\r\n'
header: Expires header: X-GPD-Session header: Cache-Control header: Date header: Vary header: Content-Length header: Content-Type header: Last-Modified header: Age header: Via header: Connection Downloaded from http://service.geopedia.world/wms/ml_aws?service=WMS&request=GetMap&styles=&format=image%2Fpng&transparent=false&version=1.1.1&layers=ttl1904&height=256&width=256&srs=EPSG:3857&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493


True

In [97]:
poly_sw_cb.set_mode('train')
downloader_train_images = BatchDownloader(root_dir+'sample_region_2/', poly_sw_cb,
                             S2Request, (WMS_INSTANCE,),
                             {'width':256, 'height':256, 'crs':3857, 'time':('2016-04-15','2016-05-15'), 'layers':'TRUE_COLOR', 'maxcc':0.3})

In [98]:
downloader_train_images.download_data()

DEBUG:root:Adding patch 52.678145335053124,5.607234184146118 --> 52.69208592394724,5.630231055419578 to queue ([(624194.4541377827, 6923684.013462493), (626754.4541377827, 6926244.013462493)])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): services.sentinel-hub.com
DEBUG:urllib3.connectionpool:http://services.sentinel-hub.com:80 "POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1" 200 9228
DEBUG:root:There are 4 urls queued for download
DEBUG:root:Adding patch 52.690692065307466,5.607234184146118 --> 52.70462864937853,5.630231055419578 to queue ([(624194.4541377827, 6925988.013462493), (626754.4541377827, 6928548.013462493)])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): services.sentinel-hub.com
DEBUG:urllib3.connectionpool:http://services.sentinel-hub.com:80 "POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1" 200 9228
DEBUG:root

send: b'POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1\r\nHost: services.sentinel-hub.com\r\nUser-Agent: python-requests/2.18.4\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Type: application/json\r\nContent-Length: 317\r\n\r\n'
send: b'{"type": "Polygon", "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:EPSG::3857"}}, "coordinates": [[[624194.4541377827, 6923684.013462493], [624194.4541377827, 6926244.013462493], [626754.4541377827, 6926244.013462493], [626754.4541377827, 6923684.013462493], [624194.4541377827, 6923684.013462493]]]}'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date header: Server header: Content-Length header: Content-Type header: Via header: Connection Downloaded from http://services.sentinel-hub.com/index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59
send: b'POST /index/v2/search?&maxcount=10000&maxcc=0.3&tim

DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): services.sentinel-hub.com
DEBUG:urllib3.connectionpool:http://services.sentinel-hub.com:80 "POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1" 200 9228
DEBUG:root:There are 4 urls queued for download
DEBUG:root:Adding patch 52.71577471315601,5.607234184146118 --> 52.72970328902998,5.630231055419578 to queue ([(624194.4541377827, 6930596.013462493), (626754.4541377827, 6933156.013462493)])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): services.sentinel-hub.com
DEBUG:urllib3.connectionpool:http://services.sentinel-hub.com:80 "POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1" 200 9228
DEBUG:root:There are 4 urls queued for download
DEBUG:root:Adding patch 52.72831063161974,5.607234184146118 --> 52.74223520412093,5.630231055419578 to queue ([(624194.4541377827, 6932900.013462493), (6267

send: b'POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1\r\nHost: services.sentinel-hub.com\r\nUser-Agent: python-requests/2.18.4\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Type: application/json\r\nContent-Length: 317\r\n\r\n'
send: b'{"type": "Polygon", "crs": {"type": "name", "properties": {"name": "urn:ogc:def:crs:EPSG::3857"}}, "coordinates": [[[624194.4541377827, 6928292.013462493], [624194.4541377827, 6930852.013462493], [626754.4541377827, 6930852.013462493], [626754.4541377827, 6928292.013462493], [624194.4541377827, 6928292.013462493]]]}'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date header: Server header: Content-Length header: Content-Type header: Via header: Connection Downloaded from http://services.sentinel-hub.com/index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59
send: b'POST /index/v2/search?&maxcount=10000&maxcc=0.3&tim

DEBUG:urllib3.connectionpool:http://services.sentinel-hub.com:80 "POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1" 200 9228
DEBUG:root:There are 4 urls queued for download
DEBUG:root:Adding patch 52.74084294702376,5.607234184146118 --> 52.75476351663683,5.630231055419578 to queue ([(624194.4541377827, 6935204.013462493), (626754.4541377827, 6937764.013462493)])
DEBUG:urllib3.connectionpool:Starting new HTTP connection (1): services.sentinel-hub.com
DEBUG:urllib3.connectionpool:http://services.sentinel-hub.com:80 "POST /index/v2/search?&maxcount=10000&maxcc=0.3&timefrom=2016-04-15T00:00:00&timeto=2016-05-15T23:59:59 HTTP/1.1" 200 9228
DEBUG:root:There are 4 urls queued for download
DEBUG:root:Adding patch 52.7533716598042,5.607234184146118 --> 52.76728822701448,5.630231055419578 to queue ([(624194.4541377827, 6937508.013462493), (626754.4541377827, 6940068.013462493)])
DEBUG:urllib3.connectionpool:Starting new HTTP connect

...

DEBUG:urllib3.connectionpool:https://services.sentinel-hub.com:443 "GET /ogc/wms/71513b0b-264d-494a-b8c4-c3c36433db28?showLogo=false&service=WMS&request=GetMap&layers=TRUE_COLOR&format=image/png&version=1.3.0&height=256&width=256&crs=EPSG:3857&maxcc=30&time=2016-05-08T10:40:27/2016-05-08T10:40:27&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493 HTTP/1.1" 200 None
DEBUG:root:(<gather_data.Patch object at 0x1127e9f60>, <requestDownload.DownloadRequest object at 0x1128f2898>)
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): services.sentinel-hub.com


reply: 'HTTP/1.1 200 OK\r\n'
header: Access-Control-Allow-Credentials header: Access-Control-Allow-Headers header: Access-Control-Allow-Methods header: Access-Control-Allow-Origin header: Content-Type header: Date header: Server header: transfer-encoding header: Connection Downloaded from https://services.sentinel-hub.com/ogc/wms/71513b0b-264d-494a-b8c4-c3c36433db28?showLogo=false&service=WMS&request=GetMap&layers=TRUE_COLOR&format=image/png&version=1.3.0&height=256&width=256&crs=EPSG:3857&maxcc=30&time=2016-05-08T10:40:27/2016-05-08T10:40:27&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493
send: b'GET /ogc/wms/71513b0b-264d-494a-b8c4-c3c36433db28?showLogo=false&service=WMS&request=GetMap&layers=TRUE_COLOR&format=image/png&version=1.3.0&height=256&width=256&crs=EPSG:3857&maxcc=30&time=2016-05-11T10:53:43/2016-05-11T10:53:43&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493 HTTP/1.1\r\nHost: services.sentinel-hub.com\r\nUser-Agent: 

DEBUG:urllib3.connectionpool:https://services.sentinel-hub.com:443 "GET /ogc/wms/71513b0b-264d-494a-b8c4-c3c36433db28?showLogo=false&service=WMS&request=GetMap&layers=TRUE_COLOR&format=image/png&version=1.3.0&height=256&width=256&crs=EPSG:3857&maxcc=30&time=2016-05-11T10:53:43/2016-05-11T10:53:43&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493 HTTP/1.1" 200 None
DEBUG:root:(<gather_data.Patch object at 0x1127e9f60>, <requestDownload.DownloadRequest object at 0x1128f2198>)


reply: 'HTTP/1.1 200 OK\r\n'
header: Access-Control-Allow-Credentials header: Access-Control-Allow-Headers header: Access-Control-Allow-Methods header: Access-Control-Allow-Origin header: Content-Type header: Date header: Server header: transfer-encoding header: Connection Downloaded from https://services.sentinel-hub.com/ogc/wms/71513b0b-264d-494a-b8c4-c3c36433db28?showLogo=false&service=WMS&request=GetMap&layers=TRUE_COLOR&format=image/png&version=1.3.0&height=256&width=256&crs=EPSG:3857&maxcc=30&time=2016-05-11T10:53:43/2016-05-11T10:53:43&bbox=642626.4541377827,6939812.013462493,645186.4541377827,6942372.013462493


True